In [ ]:
# Perform Synthesizer Parameter Search

In [ ]:
import IPython.display as ipd

from ohc.search.search import EvolutionarySearch
from ohc.search.search import FindMySound
from ohc.vst import VSTHost
from ohc.fitness.fitness import FitnessFunction
from ohc.fitness.clap import CLAPSimilarity

%load_ext autoreload
%autoreload 2

In [ ]:
PLUGIN_PATH = "/Library/Audio/Plug-Ins/VST3/Dexed.vst3"
SAMPLE_RATE = 48000
BLOCK_SIZE = 512

In [ ]:
# Create a VST
vst = VSTHost(PLUGIN_PATH, "fixed", SAMPLE_RATE, BLOCK_SIZE)

# Select parameters to search over
params = vst.list_params(filter_midi_cc=True)
active_params = [p for p in params if p != "Bypass"]
vst.set_active_params(active_params)

In [ ]:
# Initialize the CLAP embedding
clap = CLAPSimilarity(sample_rate=SAMPLE_RATE)

In [ ]:
# Create a fitness function
text_target = ["toy piano"]
fitness = FitnessFunction(vst, clap, text_target=text_target, clap_batch_size=128)

In [ ]:
problem = FindMySound(
    "max",
    num_params=len(active_params),
    min_value=0.0,
    max_value=1.0,
    fitness_function=fitness,
)

In [ ]:
search = EvolutionarySearch(problem, population_size=512, max_generations=10, logger="custom")

In [ ]:
search.run()

In [ ]:
best_discovered_solution = search._searcher.status["best"]

In [ ]:
audio, _ = vst.render_now(best_discovered_solution.values.unsqueeze(0), 48, 1.0, 1.0)[0]

In [ ]:
ipd.Audio(audio, rate=SAMPLE_RATE)